# ***<font color="RoyalBlue">SNP解析 </font>*** <font color="Tomato"> Quick Start </font>


##### _E. coli_ O157:H7 Sakai株を参照配列に、全てデフォルトのパラメーターでSNP解析する方法です。

<br>

***

# ***<u><font color="RoyalBlue">解析手順</font></u>***

<font color="DarkOrange">※ 解析前にデータのQuality check(grape_qc_asembly)を行うことを推奨</font>

## 1. 入力ファイルのアップロード

解析プロジェクトのフォルダへ、FASTQファイルをアップロードしてください。  
左サイドバー上部の⬆︎ボタンから、または、解析プロジェクトフォルダを開きファイルをドラック＆ドロップでアップロードできます。
<br>


使用できるファイルの拡張子は、`fastq.gz`です。<br>
ファイル名は、菌株名で始まるようにしてください。<br>
たとえば、以下のようなフォーマットに対応しています。

- イルミナフォーマット（e.g., strain_S00_L001_R1_001.fastq.gz）
- シンプルフォーマット（e.g., strain_R1.fastq.gz）


## 2. 菌株リストの作成

次のいずれかの方法で、菌株リストを作成します。

### 2.1. Jupyter Labで作成する
- 左サイドバー上部の「 + 」ボタン→「Other」の「Text file」を押下し、ファイルを新規作成できる。
- 編集が完了したら、Ctrl＋s（Mac:Command+s）を押下し、ファイル名を設定して保存する。
### 2.2.  Notepad、メモ帳、Vim等(何でも良い)で作成して、解析フォルダにコピーする。
- Windowsで作成すると、正常に動かない可能性があります。（改行コードの違いのため）
- LinuxまたはMacで作成してください。
### 2.3. 下記コマンドで作成する。       
- 「file_name = 」の後に、リスト名をダブルクオート(")で囲って入力してください（例, "ファイル名") 
- 「user_input = ("""」の後に、菌株リストを入力してください（1行に1つの株名）。

<font color="Tomato">────────────── 　↓↓↓ 　***コマンド実行***　 ↓↓↓　 ──────────────</font>

In [ ]:
# リストのファイル名を入力
file_name = "list.txt"

# 菌株リストを貼り付ける
user_input = ("""
A0001
A0002
A0003
A0004
A0005
A0006
A0007
A0008
"""
)

# 前後の改行を除去後に改行追加
user_input = user_input.strip("\n") + "\n"

# 入力された要素をファイルに書き込む
with open(file_name, "w") as file:
    file.write(user_input)
!echo 'Complete!'

<font color="Tomato">────────────────────────────────────────────</font>
<br>

## 3. FASTQリストの作成

2.で作成した菌株リスト中の各菌株に紐づくFASTQファイル名(ショートリードデータのファイル名)一覧の作成を以下の手順で行います。

### 3.1. find_strain_pairs.pyの実行
- 「file_name = 」の後に、前の項で作成した株名リスト名をダブルクオート(")で囲って入力してください。 
- `パラメータを設定`に記載されている内容で修正したいものがあれば修正し、実行してください。
    - 記載されている内容のまま実行することも可能です。
    
<font color="Tomato">────────────── 　↓↓↓ 　***コマンド実行***　 ↓↓↓　 ──────────────</font>

In [ ]:
import subprocess

####################################################
# パラメータを設定
####################################################
# 2.で作成した菌株リストのファイル名を入力
file_name = "list.txt"
# FASTQファイルの拡張子を入力
file_extension = "fastq.gz"
# 作成するFASTQリストのファイル名
fastq_list_name = "list_fastq.tsv"
# ペアが見つからなかった菌株一覧のファイル名
unpaired_list = "unpaired_fastq.tsv"

####################################################
# Run find_strain_pairs.py
# ※以下は変更しないでください※
####################################################
command = [
    "find_strain_pairs.py", file_name,
    "--file_extension", file_extension,
    "--paired_list", fastq_list_name,
    "--unpaired_list", unpaired_list
]
subprocess.run(command, capture_output=False, text=True)
!echo 'Complete!'

<font color="Tomato">────────────────────────────────────────────</font>
<br>

### 3.2. FASTQリストの確認・修正
- `list_fastq.tsv`(FASTQリスト)を開いて確認し、各菌株名に対し2つのFASTQファイル(Read1と2)名が正しく記載されていることを確認します。
- `unpaired_fastq.tsv`を開いて確認し、FASTQファイルのペアが見つからなかった菌株がないか確認します。
    - あった場合、`list_fastq.tsv`にその菌株名をコピーして、タブ区切りでFASTQファイルのペアを記載してください。
- FASTQリストの記載例は以下の通りです(各列はタブ区切り、改行コードは\n)。

```plaintxt
strain1	strain1_R1.fastq.gz	strain1_R2.fastq.gz
strain2	strain2_R1.fastq.gz	strain2_R2.fastq.gz
```

## 4. SNPcasterの実行
<font color="DarkOrange">※ 既にSNPcasterを行った株がある場合は、SNPフォルダ（株名のフォルダ）を実行フォルダに移動する。その株についてはBactSNPはスキップされ、SNPフォルダがない株においてのみBactSNPが実行される。<br>
</font>

<br>

下記セル内のリスト部分（「list = "list.txt"」のダブルクオートで囲まれた部分）を作成したリスト名に変えて、実行する。

#### オプション
    *オプショナル。指定しなければ、デフォルト値で実行される。
    ※clustered SNPは、組換え領域と考えてSNPから除去する領域。

|パラメータ|説明|
|---|---|
|-r|参照配列のファイル名（例．ref.fasta）|
|-i|菌株リスト（例．list）|
|-a*|BactSNPの--allele_freqの値(デフォルトは0.9)|
|-c*|隣接SNP (clustered SNP) で削除したい間隔(0を指定すると削除しない。デフォルトは0)|
|-d*|マスクファイル(指定しない場合はマスク処理を行わない)|
|-f*|fastq listのパス(BactSNPを実行する菌株がある場合は必須)|
|-g*|gubbins実行するか否か (1:実行、0:実行しない) (デフォルトは0) |
|-j*|BactSNPのジョブ数（デフォルトは4） |
|-t*|スレッド数 (デフォルトは8) |

#### ファイルフォーマット
fastq listは以下のように菌株名とそれに紐づくRead1・2ファイル名を記載したtsv(タブ区切り)ファイルを指定する。
```
strain1 strain1_R1.fastq.gz strain1_R2.fastq.gz
strain2 strain2_R1.fastq.gz strain2_R2.fastq.gz
```

<br>

<font color="Tomato">────────────── 　↓↓↓ 　***コマンド実行***　 ↓↓↓　 ──────────────</font>

In [ ]:
####################################################
# パラメータを設定
####################################################
# 参照配列ファイル
reference = '../../sample_data/ehec/Sakai_BA000007.fasta'
# 菌株リストファイル
list = "list.txt"
# BactSNPのアレル頻度(--allele_freqの値)
allele_freq = 0.9
# 隣接SNP (clustered SNP) で削除したい間隔
cluster = 0
# gubbins実行する=1、しない=0
gubbins = 0
# BactSNPのジョブ数
jobs = 4
# スレッド数
threads = 8
# fastq リストを使用しない場合、以下のfastq_listの前に#を追加してください
fastq_list = "list_fastq.tsv"
# mask処理を行う場合、以下の行頭の#を削除し、そのファイルパスを""の間に入力してください
#mask = "../../sample_data/ehec/mask.tsv"

####################################################
# Run SNPcaster
####################################################
# 
extra_options = ""
extra_options += f"-f {fastq_list} " if 'fastq_list' in locals() and fastq_list else ""
extra_options += f"-d {mask} " if 'mask' in locals() and mask else ""

!bash snpcaster.sh \
    -r $reference \
    -i $list \
    -a $allele_freq \
    -c $cluster \
    -g $gubbins \
    -j $jobs \
    -t $threads $extra_options
!echo 'Complete!'

<font color="Tomato">────────────────────────────────────────────</font>


#### 出力されるファイルについて

実行後、snpcaster\_[日付]\_[時間]\_[リスト名]のフォルダが作製される。<br>

***<font color="Red">※
    FASTQファイルが存在しない場合、エラーが出て終了する。<br>
    FASTQファイルがない株のリストとして”missing_list”が出力される。<br></font>***

出力されるファイル、フォルダの内容や注意点等については<b>SNPcaster.ipynbの「9.2.出力」</b>の説明を参照してください。<br>


<br>

## 5. 系統樹の作製

得られたfinal_snp.fasta等を用いて、最尤法による系統樹作製（raxml-ng.sh）を実行する。<br>

### 5.1. RAxML-NGの実行
  - 下記セルで、「input=」の後に解析するSNPファイルのパスを入れてください。
  - 例. 'snpcaster_20240115_094704_list_test/3_results_without_gubbins/final_snp.fasta'<br>
    ↑上記の例で、<u><font color="Red">"20240115_094704_list_test"の部分は、解析ごとに異なります</u></font>ので注意してください。
    
  - 必要に応じて、参照配列の名前（デフォルトではRef）を変えたり、削除したりしてください。


- コマンド
  - bash snpcaster_20230821/raxml-ng.sh [input] [threads] [bootstrap]
<br>
- オプション

|パラメータ|必須か|概要|
|---|---|---|
|input|●|入力ファイル (例 xxx.fasta)。Input alignment in PHYLIP/FASTA/NEXUS/CLUSTAL/MSF format|
|threads|●|modeltest-ngのスレッド数 (例 12) ※ raxml-ngのスレッド数は自動選択|
|bootstrap|-|bootstrap (デフォルトは1000)|

<br>

<font color="Tomato">────────────── 　↓↓↓ 　***コマンド実行***　 ↓↓↓　 ──────────────</font><br>
※ ログは、実行ディレクトリの`raxml-ng.log`を参照してください。（<u><font color="Red">***実行のたびに上書きされます。***</u></font>）


In [ ]:
####################################################
# パラメータを設定
####################################################
input = 'snpcaster_20240115_094704_list_test/3_results_without_gubbins/final_snp.fasta'
# Gubbinsを実行した結果を使いたい場合は、以下を使用してください。
# 上記のinputの行を削除⇒以下の行頭の# を消す⇒'snpcaster_20240115_094704_list_test'の箇所をSNPcasterの結果フォルダ名に変更してください。
# input = "snpcaster_20240115_094704_list_test/5_results_with_gubbins/final_snp_after_gubbins.fasta"
threads = 8
bootstrap = 1000

####################################################
# Run raxml-ng
####################################################
!bash raxml-ng.sh $input $threads $bootstrap > raxml-ng.log
!echo 'Complete!'

<font color="Tomato">────────────────────────────────────────────</font>
<br>

### 5.2. 系統樹ファイルの出力先
  - RaxMLフォルダ(raxml\_results\_[日付]\_[時間])が解析フォルダに作製され、そこに系統樹ファイルが出力されます。
  - そのフォルダ内の`final_snp.fasta_bootstrap.nwk`がブートストラップ値付きの系統樹ファイルです。
<br>


### 5.3. 系統樹ファイルの確認
#### 5.3.1. 最尤法系統樹
系統樹ファイルを、MEGA、CLC Genomics Workbench等で開く。<br>
MEGAで画像保存する場合は、Image/Copy to Clipboard。MEGAでRootを含む画像保存する場合は、Rootとなる線を選んで「Place Root on Branch」を押して、「Show Subtree Separately」を使う。<br>
<br>
#### 5.3.2. ハプロタイプネットワーク図
Minimum spanning tree等のハプロタイプネットワーク図の作製にはPopARTを用いる。
snpcaster_日付_時間_list_test/3_results_without_gubbins フォルダ内のfinal_snp.nexを選択して開く。<br>
※本ファイルはsnpcasterの結果作製されるため、RAxML-NGを実行する必要はない。
